Some text

In [4]:
import sys
import os
print(sys.path)

['/Users/yotamcon/projects/gep-dynamics/notebooks', '/Users/yotamcon/projects/gep-dynamics', '/Applications/miniconda3/envs/gep-dynamics/lib/python39.zip', '/Applications/miniconda3/envs/gep-dynamics/lib/python3.9', '/Applications/miniconda3/envs/gep-dynamics/lib/python3.9/lib-dynload', '', '/Applications/miniconda3/envs/gep-dynamics/lib/python3.9/site-packages']


In [5]:
a = 1